<a href="https://colab.research.google.com/github/Ashita78/ciclo4/blob/main/heartattack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Propósitos: Proyectar factores de riesgo de corazón teniendo en cuenta
diferentes factores como estilos de vida, historia médica.



Se mostrará también diferentes tipos de diagramas para buscar información especifica del dataset.

Objetivos: Al finalizar este caso

Se comprendera la influencia de los diferentes factores en los ataques del corazón.
Se mostrarán diferentes gráficos usando plotly

Justificación

Identificar tempranamente a individuos en riesgo.
Desarrollar estrategias de prevención personalizadas.
Optimizar la asignación de recursos sanitarios.
Reducir la carga económica asociada a enfermedades cardíacas.
Mejorar la calidad de vida de la población.


Contexto analítico: Se utilizan datos en formato CSV sobre factores de riesgo del corazón utilizando este dataset https://www.kaggle.com/datasets/waqi786/heart-attack-dataset

Se pasa a importar algunas librerias para hacer los gráficos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

En Google Colab, se puede trabajar con SQLAlchemy, una librería de Python que facilita se encarga de dar acceso a bases de datos relacionales y permite realizar operaciones sobre ellas. Veamos cómo puedes usar SQLAlchemy en Colab:
1) Se Crea un Engine (Motor) para la Base de Datos:
2)  El engine es el punto de entrada a la base de datos y permite que SQLAlchemy se comunique con ella.


Sqlite es una versión ligera de SQL

In [ ]:
from sqlalchemy import create_engine
import os

# Crear el engine para la base de datos chinook.db
engine = create_engine(os.path.join('sqlite:///','data', 'heart_attack_database.db'))

In [ ]:
query = "SELECT name FROM sqlite_master WHERE type='table'"
import pandas as pd
df = pd.read_sql(query, con=engine)
df

Se usa el paquete sqalchemy

Lenguaje de consulta estructurado SQL

Se va a utilizar el lenguaje de consulta estructurado ya que este permite la comunicación con las based de datos relacionales de una forma eficiente para poder extraer datos.

Se usará el paquete sqlalchemy para hacer queries (consultas) unido con pandas.

In [ ]:
import sqlalchemy
import os

version_sqlalchemy = sqlalchemy.__version__  ## Almacena como texto la versión actual de sqlalchemy
print("la versión instalada de sqlalchemy es la: ", version_sqlalchemy)

version_mayor = int(version_sqlalchemy.split(".")[0])  ## Extrae el primer número de la versión (índice 0)
print(version_mayor)

paquete = "pansql" ## por defecto trata de instalar pansql

if version_mayor >= 2:                       # Prueba si la versión mayor de sqlalchemy es igual o superior a 2
  os.system("pip install " + paquete)           # de ser así instala el paquete  "pansql"
  print("Se instaló " + paquete)                # Confirma la instalación del paquete
else:                                        # De lo contario (la versión mayor es inferior a 2)
  paquete = "pandasql"                          # cambia el paquete objetivo a "pandasql"
  os.system("pip install " + paquete)           # instala el paquete  "pandasql"
  print("Se instaló " + paquete)                # Confirma la instalación del paquete

exec('from ' + paquete +  ' import sqldf')      # Carga la función que permite ejecutar SQL con dataframes de pandas
print("Se cargó la función sqldf")

Con %sql lite se puede trabajar con base de datos SQLITE o POSTGRESQL

In [ ]:
%load_ext sql
%sql sqlite:///mi_basededatos.db
%sql SELECT * FROM mi_tabla WHERE columna = 'valor'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/',force_remount=True)

In [ ]:
#ADVERTENCIA: Comentar esta celda si NO está trabajando dentro de Google Colaboratory
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
os.getcwd()
os.chdir("/content/drive/MyDrive/Notebooks Inteligencia artificial/")


                                              Primeros pasos con el código

Caso de estudio :¿Que patrones existen entre los ataques del corazón y diferentes factores como estilo de vida e historia médica?


Los datos de heart attack dataset muestran las siguientes categorias
Gender
Age
Blood pressure
Cholesterol
Has diabetes
Smoking Status
Chest pain test
Treatment

Se pasa a leer la base de datos heart attack sacada de Kaggle

In [ ]:
# Leer el archivo CSV
df = pd.read_csv('heart_attack_dataset.csv',encoding='latin1')

# Se muestran las primeras 4 filas del DataFrame para saber de que se trata el dataset
print(df.head())

In [ ]:
# Verificar si la columna 'tratamiento' existe en el DataFrame
if 'treatment' in df.columns:
    print("La columna 'treatment' existe en el DataFrame.")

    # Identificar los tipos de tratamiento
    tipos_de_tratamiento = df['treatment'].unique()
    print("Tipos de tratamiento en el dataset:", tipos_de_tratamiento)
    print("Número de tipos de tratamiento:", len(tipos_de_tratamiento))
else:
    print("La columna 'tratamiento' no se encuentra en el DataFrame.")

In [ ]:
# Obtener estadísticas descriptivas para todas las variables numéricas
summary_stats = df.describe()
print(summary_stats)

En este código, se utiliza el método .sum() para contar cuántas veces aparece cada valor en la columna “género”. Esto nos dará la cantidad total de hombres y mujeres en el conjunto de datos.

In [ ]:
# Cuenta la cantidad de hombres y mujeres
con_hombres = (df["Gender"] == "Male").sum()
con_mujeres = (df["Gender"] == "Female").sum()

print(f"Cantidad de Hombres: {con_hombres}")
print(f"Cantidad de Mujeres: {con_mujeres}")

**Regresión logística**

Colocar las librerias necesarias, visualización libreria de datos
Para el modelo de regresión logística se va a aplicar el modelo statsmodel

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Notebooks Inteligencia artificial/heart_attack_dataset.csv")


In [ ]:
data.shape

Cuenta con mil registros y 8 variables

In [ ]:
data.info()

La mayoria de sus variables son categorias, tres variables que me indican que son de tipo entero.

In [ ]:
data.isnull().sum()

Con data isnull reviso si hay valores faltantes , se ve que no hay existencia de valores nulos.

In [ ]:
data.select_dtypes(include="O").describe()

Con data select se hace una descripción de todas las variables que son categoricas.

In [ ]:
data.select_dtypes(exclude="O").describe()

In [ ]:
target="Churn"
var_num=["Age","Blood Pressure","Cholesterol"]
var_cat=[a for a in data.columns if a not in var_num+[target,"customerID"]]

In [ ]:
var_cat

Ahora aqui se buscan las variables númericas


In [ ]:
dftarget_des=data["Treatment"].value_counts().reset_index()#*100/len(data)

In [ ]:
dftarget_des["%total"]=100*(dftarget_des["count"]/sum(dftarget_des["count"])).round(2)
dftarget_des

In [ ]:
sns.countplot(x=data["Treatment"],hue=data["Treatment"])
plt.text(-0.15,1500,s="5174\n(73%)",size=10)
plt.text(0.85,500,s="1869\n(27%)",size=10)
plt.show()

https://www.youtube.com/watch?v=BbAhzkohbws&t=166s mirar
minuto 9:09


In [ ]:
def plot_cat(df,lista,fi_col,hue=None):

    plt.figure(figsize=(20,15))
    plt.subplots_adjust(top=1.2,hspace=0.5)
    for i,j in enumerate(lista):
        plt.subplot(*fi_col,i+1)
        sns.countplot(data=df,x=j,hue=j if not bool(hue) else hue,)
        plt.xticks(rotation=45)
        plt.title(f'variable: {j}',fontsize=20)

Hacer consultas query

In [ ]:
!pip install sqlalchemy

Preprocesamiento de datos para simplificar el análisis en el futuro

En el mundo real, a menudo no podemos darnos el lujo de manejar datos perfectamente limpios y formateados. Por lo tanto, tendremos que realizar algunas operaciones en los datos proporcionados para obtener un formato que permita un análisis posterior.

Por ejemplo, la columna ```Value``` es un valor numérico, pero actualmente está en formato de cadena, lo que dificulta la realización de operaciones matemáticas en ella. Esto se debe a que a esta columna le faltan algunos valores, que se registran como "Not Available":

A la hora de escribir una `consulta compleja` como la anterior se ebe evitar escribir mucho código y mas bien adoptar una estrategia que permita automatizar cualquier cambio futuro. Por ejemplo: Si las recodificaciones cambiaran, deberíamos borrar el texto actual y sutiruirlo por uno nuevo.

Dos posibles enfoques serían:

1. Crear una tabla de equivalencias con el diccionario y usar un `INNER JOIN`

2. Crear un ciclo `for` que sustituya las tiquetas secuencialmente el lugar de toas al tiempo.



A continuación se muestra como aplicar ambos enfoques:

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
heart_attack_dataset_df =pd.DataFrame

In [ ]:
energy_short_df = pd.DataFrame(energy_short_dict.items()
                              , columns=["etiquetas_ant","etiquetas_new"])

display(energy_short_df)

In [ ]:
import csv

# Ruta al archivo CSV
# archivo_csv = '../yo/Grupo 2/heart_attack_dataset.csv'

# Inicializamos contadores
LifestyleChanges = 0
Angioplasty = 0

#with open(archivo_csv, encoding='latin1') as fichero_csv:
    #lector = csv.reader(fichero_csv)
    # Saltamos la primera línea (encabezados)
    #next(lector)

    # Recorremos las filas del archivo
    for column in lector:
        # La primera columna está en la posición 0
        valor_genero = column[7]

        if valor_genero.lower() == 'lifestyle changes':
            LifestyleChanges += 1
        elif valor_genero.lower() == 'angioplasty':
            Angioplasty += 1



print(f"Hay {LifestyleChanges} registros 'lifestyle changes' y {Angioplasty} registros 'angioplasty'.")
